In [ ]:
import math
from z3.z3 import *

from mastermind import Game, score

def color_count(code, color, n_pegs):
    return Sum([c == color for c in code])

In [ ]:
g = Game(15, 8)
g.n_colors ** g.n_pegs

In [ ]:
g.guesses = []
sol = z3.Solver()
x = [z3.Int("x[%d]" % j) for j in range(g.n_pegs)]

for i in range(g.n_pegs):
    sol.add(x[i] >= 1, x[i] <= g.n_colors)

while sol.check() == sat:
    mod = sol.model()    
    guess = [mod.eval(xi).as_long() for xi in x]
    b, w = g.guess(guess)
    print(f"{len(g.guesses):3} {guess} {b:2} {w:2}") 

    sol.add(Not(And(xi == guessi for xi, guessi in zip(x, guess))))    
        
    black = [x[i] == guess[i] for i in range(g.n_pegs)]
    sol.add(Sum(black) == b)
    
    white = []
    for color in range(1, g.n_colors + 1):
        secret_count = color_count(x, color, g.n_pegs)
        guess_count = color_count(guess, color, g.n_pegs)
        min_count = If(secret_count < guess_count, secret_count, guess_count)
        white.append(min_count)
    sol.add(Sum(white) - Sum(black) == w)

g.display()